In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from gensim.models import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/RS/glove.840B.300d.txt', binary=False,no_header=True)

In [129]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.preprocessing as preprocessing
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [130]:
main_df = pd.read_csv("/content/final26april_2_30.csv")
main_df.head()

,Unnamed: 0,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,actor1,actor2,actor3,actor4,actor5,averageRating,numVotes,plot
0,0,tt10023022,Clean,Clean,2021,94,"Action,Crime,Drama",Adrien Brody,Glenn Fleshler,Richie Merritt,Chandler DuPont,NaN,5.6,9484.0,"Clean is a garbage man, attempting a quiet lif..."
1,1,tt10059518,Unhinged,Unhinged,2020,90,"Action,Thriller",Russell Crowe,Caren Pistorius,Gabriel Bateman,Jimmi Simpson,NaN,6.0,71456.0,Academy Award winner Russell Crowe stars in Un...
2,2,tt1013743,Knight and Day,Knight and Day,2010,109,"Action,Adventure,Comedy",Tom Cruise,Cameron Diaz,Peter Sarsgaard,Jordi Mollà,NaN,6.3,201291.0,Looking forward to attending her sister April'...
3,3,tt10228134,Jolt,Jolt,2021,91,"Action,Comedy,Crime",Kate Beckinsale,Jai Courtney,Stanley Tucci,Bobby Cannavale,NaN,5.6,32967.0,A bouncer with a slightly murderous anger-mana...
4,4,tt1025100,Gemini Man,Gemini Man,2019,117,"Action,Sci-Fi,Thriller",Will Smith,Mary Elizabeth Winstead,Clive Owen,Benedict Wong,NaN,5.7,116259.0,"Retiring assassin, Henry Brogan finds himself ..."


In [131]:
main_df = main_df.dropna(subset=['plot', 'actor1', 'genres', 'primaryTitle'], axis=0, how='any')
main_df["actor2"] = main_df["actor2"].fillna("")
main_df["actor3"] = main_df["actor3"].fillna("")
main_df["actor4"] = main_df["actor4"].fillna("")

In [132]:
df = pd.DataFrame()

df["title"] = main_df["primaryTitle"]
df["genres"] = main_df["genres"]
df["plot"] = main_df["plot"]
df["imdb"] = main_df["averageRating"]
# df['actors'] = main_df['actor1']
df["actors"] = main_df.apply(lambda row: '|'.join(row[['actor1', 'actor2', 'actor3', 'actor4']]), axis=1)

df.head()

,title,genres,plot,imdb,actors
0,Clean,"Action,Crime,Drama","Clean is a garbage man, attempting a quiet lif...",5.6,Adrien Brody|Glenn Fleshler|Richie Merritt|Cha...
1,Unhinged,"Action,Thriller",Academy Award winner Russell Crowe stars in Un...,6.0,Russell Crowe|Caren Pistorius|Gabriel Bateman|...
2,Knight and Day,"Action,Adventure,Comedy",Looking forward to attending her sister April'...,6.3,Tom Cruise|Cameron Diaz|Peter Sarsgaard|Jordi ...
3,Jolt,"Action,Comedy,Crime",A bouncer with a slightly murderous anger-mana...,5.6,Kate Beckinsale|Jai Courtney|Stanley Tucci|Bob...
4,Gemini Man,"Action,Sci-Fi,Thriller","Retiring assassin, Henry Brogan finds himself ...",5.7,Will Smith|Mary Elizabeth Winstead|Clive Owen|...


In [133]:
df = df.replace({'NaN': np.nan})
df = df.dropna(subset=['plot', 'actors', 'genres', 'title', 'imdb'], axis=0, how='any')


In [134]:
def preprocess_text(text):
    tokens = word_tokenize(text)

    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    punctuations = ['(', ')', ':', ';', ',', '.', '!', '?', '-', '_', '@', '#', '$', '%', '^', '&', '*', '+', '=', '<', '>', '/', '\\', '{', '}', '[', ']', '|', '`', '~']
    filtered_tokens = [token for token in filtered_tokens if token not in punctuations]
    lowercase_tokens = [token.lower() for token in filtered_tokens]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in lowercase_tokens]
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text

df['preprocessed_plot'] = df['plot'].apply(lambda x: preprocess_text(x))
print(df['preprocessed_plot'].shape)


(1781,)


In [135]:
def generate_embedding_for_dataframe(df, col_name, model, embedding_size):
    embeddings = [] 
    for plot in df[col_name]:
        words = plot.split()
        word_embeddings = [model[word] for word in words if word in model.key_to_index]
        if len(word_embeddings) == 0:
            embedding = np.zeros(embedding_size)
        else:
            embedding = np.mean(word_embeddings, axis=0)[:embedding_size]
        embeddings.append(embedding)
    df['plot'] = embeddings
    return df, embeddings

plot_embeddings= np.empty((421, 300))
movie_df, plot_embeddings = generate_embedding_for_dataframe(df, 'plot', glove_model, 300)
movie_df.head()

,title,genres,plot,imdb,actors,preprocessed_plot
0,Clean,"Action,Crime,Drama","[0.023509283, 0.13700701, -0.1474769, 0.024914...",5.6,Adrien Brody|Glenn Fleshler|Richie Merritt|Cha...,clean garbage man attempting quiet life redemp...
1,Unhinged,"Action,Thriller","[0.031524554, 0.15953894, -0.11395648, 9.09829...",6.0,Russell Crowe|Caren Pistorius|Gabriel Bateman|...,academy award winner russell crowe star unhing...
2,Knight and Day,"Action,Adventure,Comedy","[0.06927499, 0.07401484, -0.08279022, -0.00184...",6.3,Tom Cruise|Cameron Diaz|Peter Sarsgaard|Jordi ...,looking forward attending sister april 's wedd...
3,Jolt,"Action,Comedy,Crime","[0.022425316, 0.109866515, -0.13565066, 0.0971...",5.6,Kate Beckinsale|Jai Courtney|Stanley Tucci|Bob...,bouncer slightly murderous anger-management pr...
4,Gemini Man,"Action,Sci-Fi,Thriller","[-0.021240294, 0.13526465, -0.1666123, -0.0879...",5.7,Will Smith|Mary Elizabeth Winstead|Clive Owen|...,retiring assassin henry brogan find pursued my...


In [136]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=240, random_state=42)
plot_embeddings_reduced = svd.fit_transform(plot_embeddings)
plot_similarity = cosine_similarity(plot_embeddings_reduced)

In [137]:
genres = list(set([genre for movie_genres in df['genres'].str.split(',') for genre in movie_genres]))
genre_matrix = np.zeros((len(movie_df), len(genres)))
for i, movie_genres in enumerate(movie_df['genres'].str.split(',')):
    for genre in movie_genres:
        j = genres.index(genre)
        genre_matrix[i, j] = 1

genre_similarity = cosine_similarity(genre_matrix)

In [148]:
def recommend_actors(movie_title, k=10):
    movie_index = df.index[df['title'] == movie_title].values[0]
    plot_similarities = plot_similarity[movie_index]
    genre_similarities = genre_similarity[movie_index]
    combined_similarities =  0.75*plot_similarities + 0.25*genre_similarities

    # +  0.25 * genre_similarities
    print(len(plot_similarities))# 
    similar_movies = np.argsort(combined_similarities)[:k]
    print(similar_movies)
    recommended_actors = set()
    for movie_index in similar_movies:
        movie_actors = movie_df.iloc[movie_index]['actors'].split('|')
        print( movie_df.iloc[movie_index]['title'])
        recommended_actors.update(movie_actors)
    return recommended_actors

recommend_actors("Once Upon a Time in Hollywood", k=10)

1781
[1731 1760  981 1166 1611  224  866 1767  251 1613]
Of Mice and Men
Marriage Story
Polynesian Power
Our Brand Is Crisis
Jeff Wayne's Musical Version of the War of the Worlds Alive on Stage! The New Generation
Space Jam: A New Legacy
Pride & Prejudice
Corazón
The Last Duel
Triple Divide


{'',
 'Adam Driver',
 'Ana de Armas',
 'Anthony Mackie',
 'Azhy Robertson',
 'Billy Bob Thornton',
 'Brenda Blethyn',
 'Cedric Joe',
 "Chris O'Dowd",
 'Demián Bichir',
 'Don Cheadle',
 'Donald Sutherland',
 'Dwayne Johnson',
 'Harriet Walter',
 'Isaac Sopoaga',
 'James Franco',
 'Jim Norton',
 'Jim Parrack',
 'Joaquim de Almeida',
 'Jodie Comer',
 'Julia Greer',
 'June Jones',
 'Keira Knightley',
 'Khris Davis',
 'LeBron James',
 'Liam Neeson',
 'Lily Osborne',
 'Mark Ruffalo',
 'Marti Pellow',
 'Matt Damon',
 'Matthew Macfadyen',
 'Melissa Troutman',
 'Michael Falzon',
 'Rich Miano',
 'Roger Wasserman',
 'Sandra Bullock',
 'Scarlett Johansson',
 'Theodore Johns'}

In [139]:
recommend_actors("Thor: Love and Thunder", k=4)

1781
[  10 1050  110  737]
Thor: Love and Thunder
Clash of the Titans
The Last Witch Hunter
Superman Returns


{'Brandon Routh',
 'Chris Hemsworth',
 'Christian Bale',
 'Elijah Wood',
 'James Marsden',
 'Jason Flemyng',
 'Kate Bosworth',
 'Kevin Spacey',
 'Liam Neeson',
 'Natalie Portman',
 'Ralph Fiennes',
 'Rose Leslie',
 'Sam Worthington',
 'Tessa Thompson',
 'Vin Diesel',
 'Ólafur Darri Ólafsson'}

In [140]:
# movie_df = movie_df.dropna(subset=['actors'], axis=0, how='any')
print(movie_df['actors'].isnull().sum())

new_movie_plot = preprocess_text("After barely surviving his grievous wounds from his mission in Dhaka, Bangladesh, Tyler Rake is back, and his team is ready to take on their next mission. Tasked with extracting a family who is at the mercy of a Georgian gangster, Tyler infiltrates one of the world's deadliest prisons in order to save them. But when the extraction gets hot, and the gangster dies in the heat of battle, his equally ruthless brother tracks down Rake and his team to Sydney, in order to get revenge.")
new_movie_genres = ['Action', 'Thriller']

new_movie_genre_encoding = np.zeros((1, len(genres)))
for genre in new_movie_genres:
    j = genres.index(genre)
    new_movie_genre_encoding[0, j] = 1

genre_similarity_new = cosine_similarity(new_movie_genre_encoding, genre_matrix)
 
words = new_movie_plot.split()
word_embeddings = [glove_model[word] for word in words if word in glove_model.key_to_index]
if len(word_embeddings) == 0:
  new_movie_plot_embedding = np.zeros(300)
else:
  new_movie_plot_embedding = np.mean(word_embeddings, axis=0)[:300]

new_movie_plot_embedding = new_movie_plot_embedding.reshape(1,-1)
new_plot_embedding_reduced = svd.transform(new_movie_plot_embedding)

plot_similarity_new = cosine_similarity(new_plot_embedding_reduced, plot_embeddings_reduced)

combined_similarity_new = 0.75 * plot_similarity_new + 0.25 * genre_similarity_new
similar_movies = np.argsort(-combined_similarity_new)[:4]


recommended_actors = set()
liss = []

# recommended_actors = {actor for movie_index in similar_movies for actor in movie_df.iloc[movie_index]['actors'].str.split('|')}
for movie_index in similar_movies:
    print(movie_df.iloc[movie_index]['title'])
    movie_actors = movie_df.iloc[movie_index]['actors'].str.split('|')
    print(type(movie_actors))
    print(movie_actors)
    liss.append(movie_actors)
    for act in movie_actors:
    #  print(act)
      recommended_actors.update(act)


print(recommended_actors)




0
446              Extraction
269            The Virtuoso
1823          Trauma Center
103                Vendetta
1316                 2 Guns
               ...         
1811                Corazón
1210    Our Brand Is Crisis
1804         Marriage Story
1025       Polynesian Power
1775        Of Mice and Men
Name: title, Length: 1781, dtype: object
<class 'pandas.core.series.Series'>
446     [Chris Hemsworth, Bryon Lerum, Ryder Lerum, Ru...
269     [Anson Mount, Abbie Cornish, Anthony Hopkins, ...
1823    [Nicky Whelan, Bruce Willis, Tito Ortiz, Texas...
103     [Bruce Willis, Thomas Jane, Theo Rossi, Lauren...
1316    [Denzel Washington, Mark Wahlberg, Paula Patto...
                              ...                        
1811    [Ana de Armas, Demián Bichir, Roger Wasserman,...
1210    [Sandra Bullock, Billy Bob Thornton, Anthony M...
1804    [Adam Driver, Scarlett Johansson, Julia Greer,...
1025    [Dwayne Johnson, June Jones, Rich Miano, Isaac...
1775    [James Franco, Chris O'Do

In [141]:
# # import numpy as np
# # import tensorflow as tf
# # from tensorflow.keras.models import Sequential
# # from tensorflow.keras.layers import Dense, Reshape

# # Define the neural network architecture
# model = Sequential([
#     Reshape((721, 721, 1), input_shape=(721, 721)),
#     Dense(16, activation='relu'),
#     Dense(8, activation='relu'),
#     Dense(1)
# ])


# # Define the loss function and optimizer
# loss_fn = tf.keras.losses.MeanSquaredError()
# optimizer = tf.keras.optimizers.Adam()

# # Define a function to compute the similarity weights for a given input movie
# def compute_weights(movie_index):
#     # Extract the plot and genre similarities for the input movie
#     plot_similarity = plot_similarity[movie_index]
#     genre_similarity = genre_similarity[movie_index]

#     # Normalize the similarities to be between 0 and 1
#     plot_similarity /= np.max(plot_similarity)
#     genre_similarity /= np.max(genre_similarity)

#     # Reshape the similarities to have shape (721, 721, 1)
#     plot_similarity = np.reshape(plot_similarity, (721, 721, 1))
#     genre_similarity = np.reshape(genre_similarity, (721, 721, 1))

#     # Stack the similarities along the last dimension to create an input tensor with shape (721, 721, 2)
#     similarities = np.concatenate([plot_similarity, genre_similarity], axis=-1)

#     # Compute the weights using the neural network
#     weights = model(similarities).numpy().squeeze()

#     # Normalize the weights to sum to 1
#     weights /= np.sum(weights)

#     return weights

# # Define a function to compute the loss for a given set of weights
# def compute_loss(weights):
#     # Compute the combined similarity using the specified weights
#     weights = tf.reshape(weights, shape=(1, 1, 2))
#     combined_similarity = weights[..., 0] * plot_similarity + weights[..., 1] * genre_similarity
#     # combined_similarity = weights[0] * plot_similarity + weights[1] * genre_similarity

#     # Sort the movies by their similarity to the input movie
#     similar_movies = np.argsort(-combined_similarity)[:10]

#     # Recommend the actors who appear in the top-rated movies
#     recommended_actors = set()
#     for movie_index in similar_movies:
#         movie_actors = movie_df.iloc[movie_index]['actor1'].split('|')
#         recommended_actors.update(movie_actors)

#     # Compute the error between the predicted and true recommended actors
#     true_actors = set(movie_df.iloc[movie_index]['target_actors'].split('|'))
#     error = len(recommended_actors.symmetric_difference(true_actors))

#     return error

# # Train the neural network to learn the optimal weights
# num_iterations = 400
# for i in range(num_iterations):
#     with tf.GradientTape() as tape:
#         # Compute the loss for the current weights
#         weights = model.trainable_variables
#         loss = compute_loss(weights)

#     # Compute the gradients of the loss with respect to the weights
#     gradients = tape.gradient(loss, weights)

#     # Update the weights using the optimizer
#     optimizer.apply_gradients(zip(gradients, weights))